## Data

In [7]:
import pandas as pd
import numpy as np
import gc
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Load data files
hh_demand_12h_A = pd.read_csv('hh_demand_12h_A.csv')
hh_demand_true_A = pd.read_csv('hh_demand_true_A.csv')
Munich_weather_1min = pd.read_csv('Munich_weather_1min.csv')
Munich_weather_12h = pd.read_csv('Munich_weather_12h.csv')
dataA_params = pd.read_csv('DataA_params.csv', delimiter=';')

# Extract parameters
params = {row['Parameter']: float(row['Value']) for _, row in dataA_params.iterrows()}
params['c_M_PV'] = params.pop('c_MPV')  # Correct the parameter name

# Constants
alpha = params['alpha']
beta = params['beta']
gamma = params['gamma']
eta_B = params['eta_B']
eta_ref = params['eta_ref']
c_B = params['c_B']
c_M_PV = params['c_M_PV']
c_PV = params['c_PV']
G_ref = params['G_ref']
h = params['h']
i_INV = params['i_INV']
p_EL = params['p_EL']
p_FI = params['p_FI']
T_cref = params['T_cref']

# Prepare weather data
Munich_weather_1min['Timestamp'] = pd.to_datetime(Munich_weather_1min['Unnamed: 0'])
Munich_weather_1min.set_index('Timestamp', inplace=True)
Munich_weather_1min.drop(columns=['Unnamed: 0'], inplace=True)

time_range = pd.date_range(start='2013-01-01', end='2013-12-31 23:59:00', freq='T')
Munich_weather_12h['Timestamp'] = time_range[::720]  # Every 12 hours
Munich_weather_12h.set_index('Timestamp', inplace=True)
Munich_weather_12h.drop(columns=['Unnamed: 0'], inplace=True)

# Interpolation
Munich_weather_1min_full = Munich_weather_12h.reindex(time_range).interpolate(method='linear')

# Replace the 3 weeks with the actual 1-minute data provided
actual_data_periods = Munich_weather_1min.index

for period in actual_data_periods:
    Munich_weather_1min_full.loc[period] = Munich_weather_1min.loc[period]

# Free up memory
del Munich_weather_12h
gc.collect()

# Verify data lengths
print("Length of interpolated weather data:", len(Munich_weather_1min_full))
print("Length of household load data:", len(hh_demand_true_A))

# Add seasonal information
def get_season(date):
    if date.month in [12, 1, 2]:
        return 'winter'
    elif date.month in [3, 4, 5]:
        return 'spring'
    elif date.month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

Munich_weather_1min_full['Season'] = Munich_weather_1min_full.index.to_series().apply(get_season)
Munich_weather_1min_full = pd.get_dummies(Munich_weather_1min_full, columns=['Season'])
Munich_weather_1min_full['Hour'] = Munich_weather_1min_full.index.hour
Munich_weather_1min_full['Day'] = Munich_weather_1min_full.index.dayofyear

# Prepare data for the ML model
X = Munich_weather_1min_full[['Hour', 'Day', 'Season_autumn', 'Season_spring', 'Season_summer', 'Season_winter']]
y_G = Munich_weather_1min_full['G_Gk']
y_T = Munich_weather_1min_full['Ta']

# Split the data into training and testing sets
X_train, X_test, y_G_train, y_G_test = train_test_split(X, y_G, test_size=0.2, random_state=42)
X_train_rf, X_test_rf, y_T_train, y_T_test = train_test_split(X, y_T, test_size=0.2, random_state=42)

# Train the Random Forest models
model_G_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_G_rf.fit(X_train, y_G_train)
model_T_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_T_rf.fit(X_train_rf, y_T_train)

# Get predictions from the Random Forest model
X_train['G_Gk_rf'] = model_G_rf.predict(X_train)
X_train['Ta_rf'] = model_T_rf.predict(X_train_rf)
X_test['G_Gk_rf'] = model_G_rf.predict(X_test)
X_test['Ta_rf'] = model_T_rf.predict(X_test_rf)

# Train the XGBoost models
model_G_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)
model_G_xgb.fit(X_train, y_G_train)
model_T_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)
model_T_xgb.fit(X_train, y_T_train)

# Predict the full year data with XGBoost
Munich_weather_1min_full['G_Gk_rf'] = model_G_rf.predict(X)
Munich_weather_1min_full['Ta_rf'] = model_T_rf.predict(X)
X['G_Gk_rf'] = Munich_weather_1min_full['G_Gk_rf']
X['Ta_rf'] = Munich_weather_1min_full['Ta_rf']
Munich_weather_1min_full['G_Gk_pred'] = model_G_xgb.predict(X)
Munich_weather_1min_full['Ta_pred'] = model_T_xgb.predict(X)

# Merge predicted weather data with household load data
merged_data = hh_demand_true_A.copy()
merged_data['G_Gk'] = Munich_weather_1min_full['G_Gk_pred'].values
merged_data['Ta'] = Munich_weather_1min_full['Ta_pred'].values


C:\Users\jihad\AppData\Local\Temp\ipykernel_17452\2769809030.py:40: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_range = pd.date_range(start='2013-01-01', end='2013-12-31 23:59:00', freq='T')


Length of interpolated weather data: 525600
Length of household load data: 525600


C:\Users\jihad\AppData\Local\Temp\ipykernel_17452\2769809030.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['G_Gk_rf'] = Munich_weather_1min_full['G_Gk_rf']
C:\Users\jihad\AppData\Local\Temp\ipykernel_17452\2769809030.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Ta_rf'] = Munich_weather_1min_full['Ta_rf']


## PV


In [22]:
import pyomo.environ as pyo

# Create a Pyomo model
model = pyo.ConcreteModel()

# Convert datetime indices to integer indices
G_Gk_pred = {i+1: v for i, v in enumerate(Munich_weather_1min_full['G_Gk_pred'])}
Ta_pred = {i+1: v for i, v in enumerate(Munich_weather_1min_full['Ta_pred'])}

# Define sets
model.T = pyo.RangeSet(1, len(G_Gk_pred))  # Time periods

# Define parameters
model.alpha = pyo.Param(initialize=params['alpha'])
model.beta = pyo.Param(initialize=params['beta'])
model.gamma = pyo.Param(initialize=params['gamma'])
model.eta_ref = pyo.Param(initialize=params['eta_ref'])
model.T_cref = pyo.Param(initialize=params['T_cref'])
model.G_ref = pyo.Param(initialize=params['G_ref'])
model.h = pyo.Param(initialize=params['h'])

# Define variables
model.CAP_PV = pyo.Var(within=pyo.NonNegativeReals)
model.P_PV = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.T_c = pyo.Var(model.T, within=pyo.Reals)
model.A_c = pyo.Var(within=pyo.NonNegativeReals)

# Define expressions for G_t and T_a_t (solar irradiance and ambient temperature)
model.G_t = pyo.Param(model.T, initialize=G_Gk_pred)
model.T_a_t = pyo.Param(model.T, initialize=Ta_pred)

# Define constraints
def Ac_constraint(model):
    return model.A_c == model.alpha**(-1) * model.CAP_PV
model.Ac_constraint = pyo.Constraint(rule=Ac_constraint)

def Tc_constraint(model, t):
    return model.T_c[t] == model.T_a_t[t] + model.h * model.G_t[t]
model.Tc_constraint = pyo.Constraint(model.T, rule=Tc_constraint)

# Define expressions
def eta_PV_expression(model, t):
    if model.G_t[t] > 0:
        return model.eta_ref * (1 - model.beta * (model.T_c[t] - model.T_cref) + model.gamma * pyo.log(model.G_t[t] / model.G_ref))
    else:
        return 0
model.eta_PV = pyo.Expression(model.T, rule=eta_PV_expression)

def P_PV_expression(model, t):
    return model.A_c * model.G_t[t] * model.eta_PV[t] / 1000
model.P_PV_expr = pyo.Expression(model.T, rule=P_PV_expression)

# Link expression to variable
def P_PV_constraint(model, t):
    return model.P_PV[t] == model.P_PV_expr[t]
model.P_PV_constraint = pyo.Constraint(model.T, rule=P_PV_constraint)

# We do not define an objective function here because it will be defined in the NPV cell


## Battery

In [24]:
import pyomo.environ as pyo

# Assuming model is already created and necessary data is imported

# Remove existing components if they exist
model.del_component('P_B')
model.del_component('SOC')
model.del_component('SOC_initial')
model.del_component('P_HH')
model.del_component('P_req')
model.del_component('P_B_constraint')
model.del_component('SOC_constraint')
model.del_component('SOC_bounds')

# Define additional variables and parameters for the battery model
model.P_B = pyo.Var(model.T, within=pyo.Reals)  # Battery power can be positive or negative
model.SOC = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, 1))

# Initial state of charge
model.SOC_initial = pyo.Param(initialize=0)

# Define the household power demand P_HH based on the minute load
P_HH_dict = {i+1: v for i, v in enumerate(hh_demand_true_A['Load'])}
model.P_HH = pyo.Param(model.T, initialize=P_HH_dict)

# Define expressions for required power
def P_req_expression(model, t):
    return model.P_HH[t] - model.P_PV[t]
model.P_req = pyo.Expression(model.T, rule=P_req_expression)

# Define battery power constraints
def P_B_constraint(model, t):
    if t == 1:
        SOC_prev = model.SOC_initial
    else:
        SOC_prev = model.SOC[t-1]
    
    if model.P_req[t] < 0:
        return model.P_B[t] == -min(model.CAP_B * 60 / model.h ,(1 - SOC_prev) * model.CAP_B * 60 / model.h / model.eta_B, -model.P_req[t])
    else:
        return model.P_B[t] == min(model.CAP_B * 60 / model.h ,  SOC_prev * model.eta_B * model.CAP_B * 60 / model.h, model.P_req[t])
model.P_B_constraint = pyo.Constraint(model.T, rule=P_B_constraint)

# Define SOC constraints
def SOC_constraint(model, t):
    if t == 1:
        SOC_prev = model.SOC_initial
    else:
        SOC_prev = model.SOC[t-1]
    
    if model.P_B[t] > 0:
        return model.SOC[t] == SOC_prev - 1 / (model.eta_B * model.CAP_B * 60 / model.h) * model.P_B[t]
    else:
        return model.SOC[t] == SOC_prev - model.eta_B / (model.CAP_B * 60 / model.h) * model.P_B[t]
model.SOC_constraint = pyo.Constraint(model.T, rule=SOC_constraint)

# Ensure SOC is within bounds
def SOC_bounds(model, t):
    return pyo.inequality(0, model.SOC[t], 1)
model.SOC_bounds = pyo.Constraint(model.T, rule=SOC_bounds)

ERROR: Rule failed when generating expression for Constraint P_B_constraint
with index 1: PyomoException: Cannot convert non-constant Pyomo expression
(0.1330928345488582  <  0) to bool. This error is usually caused by using a
Var, unit, or mutable Param in a Boolean context such as an "if" statement, or
when checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'P_B_constraint' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression
        (0.1330928345488582  <  0) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...    

PyomoException: Cannot convert non-constant Pyomo expression (0.1330928345488582  <  0) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## NPV

In [ ]:
import pyomo.environ as pyo

# Define additional parameters for costs and rates
model.c_PV = pyo.Param(initialize=params['c_PV'])
model.c_B = pyo.Param(initialize=params['c_B'])
model.c_M_PV = pyo.Param(initialize=params['c_M_PV'])
model.p_EL = pyo.Param(initialize=params['p_EL'])
model.p_FI = pyo.Param(initialize=params['p_FI'])
model.i_INV = pyo.Param(initialize=params['i_INV'])

# Define expressions for power drawn from or fed to the grid
def P_D_expression(model, t):
    return model.P_HH[t] - model.P_PV[t] - model.P_B[t]
model.P_D = pyo.Expression(model.T, rule=P_D_expression)

# Define expressions for the aggregated quantities
def E_HH_T_expression(model):
    return sum(model.P_HH[t] for t in model.T)
model.E_HH_T = pyo.Expression(rule=E_HH_T_expression)

def E_D_plus_T_expression(model):
    return sum(model.P_D[t] for t in model.T if model.P_D[t] > 0)
model.E_D_plus_T = pyo.Expression(rule=E_D_plus_T_expression)

def E_D_minus_T_expression(model):
    return sum(-model.P_D[t] for t in model.T if model.P_D[t] < 0)
model.E_D_minus_T = pyo.Expression(rule=E_D_minus_T_expression)

# Define initial investments
def C_0_PV_expression(model):
    return model.c_PV * model.CAP_PV
model.C_0_PV = pyo.Expression(rule=C_0_PV_expression)

def C_0_B_expression(model):
    return model.c_B * model.CAP_B
model.C_0_B = pyo.Expression(rule=C_0_B_expression)

# Define annual revenue
def R_T_expression(model):
    return model.E_D_minus_T * model.p_FI + (model.E_HH_T - model.E_D_plus_T) * model.p_EL
model.R_T = pyo.Expression(rule=R_T_expression)

# Define NPV calculation
def NPV_expression(model):
    return -model.C_0_PV - model.C_0_B + sum((model.R_T - model.C_0_PV * model.c_M_PV) / (1 + model.i_INV)**t for t in range(1, 21))
model.NPV = pyo.Expression(rule=NPV_expression)

# Objective function to maximize NPV
model.objective = pyo.Objective(expr=model.NPV, sense=pyo.maximize)


## Solver

In [ ]:
import os
import pyomo.environ as pyo

# Set up the NEOS server email
os.environ['NEOS_EMAIL'] = 'jihad.jundi@tum.de'

# Solve the model using the NEOS server
solver_manager = pyo.SolverManagerFactory('neos')
results = solver_manager.solve(model, opt="ipopt", tee=True)

# Display results
model.display()

# Extract and print optimal values
optimal_CAP_PV = pyo.value(model.CAP_PV)
optimal_CAP_B = pyo.value(model.CAP_B)
optimal_NPV = pyo.value(model.NPV)

print(f"Optimal Capacity for PV (CAP_PV) in kWp: {optimal_CAP_PV}")
print(f"Optimal Capacity for Battery (CAP_B) in kWh: {optimal_CAP_B}")
print(f"Optimal Net Present Value (NPV): {optimal_NPV}")


In [ ]:
print(hh_demand_true_A.columns)
